In [5]:
import os
from dotenv import load_dotenv
import httpx # for timeout
from openai import OpenAI
import json 
from datetime import datetime

load_dotenv()

client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
    timeout=httpx.Timeout(60.0, read=5.0, write=10.0, connect=2.0),
)

# File Management

In [4]:
# list files from openai
files = client.files.list()

SyncCursorPage[FileObject](data=[FileObject(id='file-dgYnQD9kbKmjuPXXk0HujFoK', bytes=4300, created_at=1730240749, filename='image.png', object='file', purpose='vision', status='processed', status_details=None), FileObject(id='file-V7M8h3EJbvuvqTXa0inT66xz', bytes=4306, created_at=1730240733, filename='image.png', object='file', purpose='vision', status='processed', status_details=None), FileObject(id='file-QIg0cabSgj7qWeL8nU7AixNt', bytes=4337, created_at=1730240677, filename='image.png', object='file', purpose='vision', status='processed', status_details=None), FileObject(id='file-PZSXxDZyHI64Rd5jetUowEaO', bytes=4102, created_at=1730240385, filename='image.png', object='file', purpose='vision', status='processed', status_details=None)], object='list', has_more=False, first_id='file-dgYnQD9kbKmjuPXXk0HujFoK', last_id='file-PZSXxDZyHI64Rd5jetUowEaO')


In [29]:
for file in files:
    print(file.id, file.filename, file.purpose)

file-4jPm4cKXYFkjfr1nieGx3H test.json assistants
file-dgYnQD9kbKmjuPXXk0HujFoK image.png vision
file-V7M8h3EJbvuvqTXa0inT66xz image.png vision
file-QIg0cabSgj7qWeL8nU7AixNt image.png vision
file-PZSXxDZyHI64Rd5jetUowEaO image.png vision


In [30]:
file = client.files.create(
  file=open("assets/100trends2025.json", "rb"),
  purpose="assistants"
)

print(file.id, file.filename, file.purpose)

file-99Qs3tSnYJPAEDf6nM8cyv 100trends2025.json assistants


# Vector Management

In [82]:
from datetime import datetime

vector_store = client.beta.vector_stores.list()

for vector in vector_store:
    created_at = datetime.fromtimestamp(vector.created_at)
    print(vector.id, vector.name, created_at)

vs_YNXK2v63BaFzie6kV0Xelelc Future100 2025-01-20 18:24:16
vs_uIAvCIL1cEXRA4inAaMKlCIt Product Documentation 2025-01-20 16:00:47


## Add Vector

In [31]:
vector_store = client.beta.vector_stores.create(name="Future100",
    file_ids=['file-99Qs3tSnYJPAEDf6nM8cyv']
)
vector_store

VectorStore(id='vs_YNXK2v63BaFzie6kV0Xelelc', created_at=1737393856, file_counts=FileCounts(cancelled=0, completed=0, failed=0, in_progress=1, total=1), last_active_at=1737393856, metadata={}, name='Future100', object='vector_store', status='in_progress', usage_bytes=0, expires_after=None, expires_at=None)

## Remove Vector

In [59]:
client.beta.vector_stores.delete("vs_XlEgyUMktGHP5ttXCDcMJrkj")

# Assi Management

## List Assis

In [6]:
assistents = client.beta.assistants.list()

for assistant in assistents:
    created_at = datetime.fromtimestamp(assistant.created_at)
    print(assistant.id, assistant.name, created_at)

asst_7F9Oqwj9llDxP2XTggCokcV9 Shoe Example for Troubleshooting 2025-02-26 09:28:12
asst_8ttxvZV4Vrv79yL6taFAI7Qb FordConfigurationAssistant (staging, streaming json) 2025-02-13 09:24:19
asst_3ptrIJRIDAPVh6kKwVTd66gr FordConfigurationAssistant (staging, stream) 2025-02-12 17:14:38
asst_IZQvpqAlLvE6i6L0CqV6RiW2 FordConfigurationAssistant (staging) 2025-02-11 11:38:10
asst_H9fW8ntBm1U4FuY20MgDdyKO Future100 (staging) 2025-01-23 11:05:00
asst_ci9OxadkpvmJRAUwI8mOOe66 Future100 (live) 2025-01-21 11:50:52
asst_pGsOaepVL4000iJbMP7zDtXB Future100 (json response) 2025-01-20 19:45:17
asst_2lohHLJlMmkPKchnQaNS9Ztz pmmd 2024-10-29 22:37:06


## Add Assi

In [275]:
instructions = '''
    Answer with a future prediction that solves 
    the mentioned challenge, based on the provided document. 
    Answer in 2 sentences. 
    Respond in JSON format following the following scheme:
        - key "response" contains the answer in plain text without a reference to the document.
        - key "number"  according to the number that refers to the answer
        - key "trend" contains the trend where you found the answer.
    If the answer is a combination of multiple locations, consider the first occurrence only.
'''

In [360]:
instructions = '''
I work in in the fast moving consumer goods ($industry). My biggest business challenge is growing sales ($challenge). What are the three most important trends for me from the data attached and why are these important for me?

FORMAT (with no additional remarks, dont provide message annotations):

<p>The three important trends in $industry in 2025 are "$trend1", "$trend2" and "$trend3".</p>
<p>(Please elaborate $trend1. First, briefly explain what this trend is. Then relate it to $industry and $challenge and explain why this trend is relevant. This paragraph should be about 350 characters total.)</p>
<p>(Please elaborate $trend2. First, briefly explain what this trend is. Then relate it to $industry and $challenge and explain why this trend is relevant. This paragraph should be about 350 characters total.)</p>
<p>(Please elaborate $trend3. First, briefly explain what this trend is. Then relate it to $industry and $challenge and explain why this trend is relevant. This paragraph should be about 350 characters total.)</p>

EXAMPLE OUTPUT 1
<p>The three important trends in the telecoms industry in 2025 are: "Digital simplicity", "AI deal hunters" and "Shared realities".</p>
<p><b>Digital simplicity</b> focuses on reducing information overload and creating intuitive user interfaces. 
In telecoms, simplifying customer interactions can enhance user satisfaction by making it easier to understand plans and services, addressing the challenge of improving customer experience.</p>

<p><b>AI deal hunters use</b> artificial intelligence to personalize offers and pricing. For telecoms, this means tailoring plans based on customer usage patterns, enhancing customer experience by providing relevant and timely deals, which can lead to increased satisfaction and loyalty.</p>

<p><b>Shared realities</b> involve blending virtual and physical experiences without headsets. Telecoms can leverage this trend to offer immersive, interactive customer service and entertainment options, enhancing customer experience by providing innovative ways to engage with services and content.</p>

'''

In [8]:
instructions = '''
I work in automotive ($industry). My business challenge is growing sales ($challenge). What is the most important trend for me from the data attached and why is it important for me?

# RESPONSE FORMAT:
One particularly important trend in $industry in 2025 is "$trend1".

Please elaborate $trend1 like this: First, briefly explain what this trend is. Then relate it to $industry and $challenge and explain why this trend is relevant for this particular industry.

Then tell the user how they can use the potential of the trend - focus on marketing, customer experience and sales.

$trend1 is a most interesting trend, but $trend2 and $trend3 seem also relevant for your industry!

The whole response should be about 1000 characters. Make it interesting and surprising. Use short sentences. Use paragraph tags like the examples below. Translate your response into a differenent language when I tell you so. 

# Example 1:
<p><strong>One particularly important trend in automotive in 2025 is "Hyper-Personalization".</strong></p>
<p>Hyper-Personalization goes beyond basic demographics. It uses AI to craft unique customer experiences. Think bespoke car configurations. Imagine predictive maintenance schedules based on individual driving habits. In automotive, this is huge. It tackles your sales challenge head-on. Customers crave unique experiences. Hyper-Personalization delivers exactly that. It fosters loyalty and drives sales.</p>
<p>To leverage this trend, you can use data to tailor marketing messages. Offer personalized car features. Create dynamic pricing models. Surprise customers with proactive service reminders. This boosts customer satisfaction. It translates into increased sales.</p>
<p>Hyper-Personalization is a game-changer, but "AI-powered Design" and "Sustainable Mobility" seem also relevant for automotive!</p>

# Example 2:
<p><strong>One particularly important trend in retail in 2025 is "Digital simplicity."</strong></p>
<p>Digital simplicity is about reducing information overload for consumers. It involves creating streamlined, user-friendly digital experiences that cut through the clutter. In retail, this trend is crucial for enhancing operational efficiency. By simplifying digital interfaces and processes, retailers can improve customer interactions and reduce friction in the buying journey.</p>
For your challenge of growing efficiency, adopting digital simplicity can streamline operations and enhance customer satisfaction. It minimizes distractions, allowing customers to find products faster and make decisions more easily. This reduces cart abandonment and increases conversion rates.
To leverage this trend, focus on decluttering your online platforms. Simplify navigation and reduce unnecessary content. Use clean design and intuitive layouts. Optimize mobile experiences for seamless shopping. Enhance customer experience by providing clear, concise product information. Implement smart search functions to help customers find what they need quickly. This approach not only boosts efficiency but also enhances customer loyalty and sales.
<p>Digital simplicity is a most interesting trend, but "Prudent pricing" and "Sensory-inclusive shopping" seem also relevant for your industry!</p>
'''

In [366]:
assistant = client.beta.assistants.create(
    instructions=instructions,
    # temperature=2, # default is 1, 0 is deterministic, 2 is more creative
    name="Future100 (staging)",
    # response_format="auto", # can be used to define a json scheme
    tools=[{"type": "file_search"}],
    # metadata={"purpose": "use this assistant to test new prompts before releasing it to the actual assistant"},
    tool_resources={
        "file_search": {"vector_store_ids": ["vs_YNXK2v63BaFzie6kV0Xelelc"]}
    },
    model="gpt-4o",
)
print(assistant)


Assistant(id='asst_H9fW8ntBm1U4FuY20MgDdyKO', created_at=1737626700, description=None, instructions='\nne particularly important trend in $industry in 2025 is "$trend1".\n \nPlease elaborate $trend1 like this: First, briefly explain what this trend is. Then relate it to $industry and $challenge and explain why this trend is relevant for this particular industry.\n \nThen tell the user how they can use the potential of the trend - focus on marketing, customer experience and sales.\n \n$trend1 is a most interesting trend, but $trend2 and $trend3 seem also relevant for your industry! Curious? Download our full report.\n \nThe whole response should be about 1000 characters. Make it interesting and surprising. Use short sentences.\nExample 1:\nOne particularly important trend in automotive in 2025 is "Hyper-Personalization".\nHyper-Personalization goes beyond basic demographics. It uses AI to craft unique customer experiences. Think bespoke car configurations. Imagine predictive maintenance

## Edit Assi

In [9]:
my_updated_assistant = client.beta.assistants.update(
    "asst_H9fW8ntBm1U4FuY20MgDdyKO",
    instructions=instructions,
    name="Future100 (live)",
    tools=[{"type": "file_search"}],
    tool_resources={
        "file_search": {"vector_store_ids": ["vs_YNXK2v63BaFzie6kV0Xelelc"]}
    },
    # metadata={"purpose": "use this assistant to test new prompts before releasing it to the actual assistant"},
    response_format = "auto",
    model="gpt-4o-mini",
)

## Retrieve Assi

In [ ]:
assistant = client.beta.assistants.retrieve("asst_H9fW8ntBm1U4FuY20MgDdyKO")

print(assistant.id)
print(assistant.name)
print(assistant.model)
print(assistant.metadata)
print("------------- Instructions")
print(assistant.instructions)

asst_ci9OxadkpvmJRAUwI8mOOe66
Future100 (live)
gpt-4o-mini
{}
------------- Instructions

I work in automotive ($industry). My business challenge is growing sales ($challenge). What is the most important trend for me from the data attached and why is it important for me?

# RESPONSE FORMAT:
One particularly important trend in $industry in 2025 is "$trend1".

Please elaborate $trend1 like this: First, briefly explain what this trend is. Then relate it to $industry and $challenge and explain why this trend is relevant for this particular industry.

Then tell the user how they can use the potential of the trend - focus on marketing, customer experience and sales.

$trend1 is a most interesting trend, but $trend2 and $trend3 seem also relevant for your industry!

The whole response should be about 1000 characters. Make it interesting and surprising. Use short sentences. Use paragraph tags like the examples below.

# Example 1:
<p><strong>One particularly important trend in automotive in 20

## Remove Assi

In [280]:
client.beta.assistants.delete("asst_Y5954lRHM4BrCFbNCxhwdJtH")

AssistantDeleted(id='asst_Y5954lRHM4BrCFbNCxhwdJtH', deleted=True, object='assistant.deleted')

# Threads

In [397]:
prompt = f"I am working in `Logistics` and i am concerned about `Efficiency`."

assistant_id = "asst_H9fW8ntBm1U4FuY20MgDdyKO"

thread = client.beta.threads.create()

### Run at once

In [398]:
thread_message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=prompt,
)

In [399]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant_id,
)


In [400]:
while run.status in ["queued", "in_progress"]:
    keep_retrieving_run = client.beta.threads.runs.retrieve(
        thread_id=thread.id, run_id=run.id
    )

    # If the run status is completed
    if keep_retrieving_run.status == "completed":
        # Retrieve the assistant's response
        all_messages = client.beta.threads.messages.list(thread_id=thread.id)

        # Display the assistant's response
        print(f"prompt: {prompt}")
        print(f"response: {all_messages.data[0].content[0].text.value}")

        break
    elif keep_retrieving_run.status in ["queued", "in_progress"]:
        pass
    else:
        break


prompt: I am working in `Logistics` and i am concerned about `Efficiency`.
response: <p><strong>One particularly important trend in logistics in 2025 is "Automation and AI Integration".</strong></p>
<p>This trend involves leveraging advanced technologies like AI and robotics to streamline operations, reduce manual labor, and enhance efficiency. As logistics grapples with increasing demand for faster delivery and operational efficiency, automation can be a game-changer. It addresses the core efficiency challenge you face, as automating repetitive tasks can significantly boost productivity and reduce errors.</p>
<p>To tap into this trend, consider implementing AI-driven systems for inventory management and order fulfillment. Invest in robotics for warehousing—think autonomous vehicles and drones for quick deliveries. Train your workforce to collaborate with these technologies, enhancing customer service as a result. Employ data analytics to predict demand and optimize routes, resulting i

In [157]:
dict_response = json.loads(all_messages.data[0].content[0].text.value)
dict_response

{'response': 'In the near future, implementing an agile product development framework combined with continuous customer feedback loops will significantly streamline product development and maximize product-market fit. This approach will result in quicker iterations and enhancements, allowing sales teams in logistics to adapt rapidly to market demands.',
 'number': 1,
 'trend': 'Streamline product development and ensure product-market fit'}

### Streaming

In [273]:
thread_message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=prompt,
)

run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id="asst_pGsOaepVL4000iJbMP7zDtXB",
    stream=True
)

In [274]:
# Initialize a buffer to accumulate the final message content
message_buffer = ""

for event in run:
    event_type = getattr(event, "event", None)

    if event_type == "thread.message.created":
        pass
        # print(f"[Message Created] Message ID: {event.data.id}, Status: {event.data.status}")

    elif event_type == "thread.message.in_progress":
        pass
        # print(f"[Message In Progress] Message ID: {event.data.id}, Status: {event.data.status}")

    elif event_type == "thread.message.delta":
        # Extract the delta updates (incremental content)
        delta_content = event.data.delta.content
        for block in delta_content:
            if block.type == "text":
                message_buffer += block.text.value
                print(block.text.value, end='', flush=True)  # Print streamed text

    elif event_type == "thread.message.completed":
        print("\n[Message Completed]")
        print(f"Final Message: {message_buffer}")
        message_buffer = ""  # Reset the buffer for the next message

    else:
        pass
        # print(f"[Unhandled event] {event}")


{"response":"To address concerns around streamlining product development and ensuring product-market fit, the adoption of agile methodologies will facilitate faster adaptation to market needs and preferences. Additionally, focusing on continuous customer engagement and feedback loops will enhance alignment with market demands, promoting successful product launches.","number":2,"trend":"agile methodologies in logistics"}
[Message Completed]
Final Message: {"response":"To address concerns around streamlining product development and ensuring product-market fit, the adoption of agile methodologies will facilitate faster adaptation to market needs and preferences. Additionally, focusing on continuous customer engagement and feedback loops will enhance alignment with market demands, promoting successful product launches.","number":2,"trend":"agile methodologies in logistics"}


### Without Assistant

In [288]:
thread = client.beta.threads.create(
messages=[
  {
    "role": "user",
    "content": prompt,
    "attachments": [
      {
        "file_id": "file-99Qs3tSnYJPAEDf6nM8cyv",
        "tools": [{"type": "file_search"}]
      }
    ]
  }
]
)

In [289]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id="asst_ci9OxadkpvmJRAUwI8mOOe66",
)


In [290]:
while run.status in ["queued", "in_progress"]:
    keep_retrieving_run = client.beta.threads.runs.retrieve(
        thread_id=thread.id, run_id=run.id
    )

    # If the run status is completed
    if keep_retrieving_run.status == "completed":
        # Retrieve the assistant's response
        all_messages = client.beta.threads.messages.list(thread_id=thread.id)

        # Display the assistant's response
        print(f"prompt: {prompt}")
        print(f"response: {all_messages.data[0].content[0].text.value}")

        break
    elif keep_retrieving_run.status in ["queued", "in_progress"]:
        pass
    else:
        break


prompt: I am working in `Sales` in the industry `Logistics` and i am concerned about `Streamline product development and ensure product-market fit`.
response: The three important trends in Logistics industry in 2025 are "Creative product development", "AI integration for efficiency" and "Sustainability as a priority".

Creative product development leverages tools like generative AI for ideation and prototype creation. In logistics, this accelerates the development of solutions tailored to market needs, addressing the challenge to streamline product development and achieve product-market fit【4:0†source】.

AI integration for efficiency involves utilizing artificial intelligence to enhance operations. In logistics, AI can optimize routes, forecast demand more accurately, and streamline supply chain processes, ensuring products meet market requirements swiftly, thus aiding in aligning product-market fit【4:6†source】.

Sustainability as a priority emphasizes reducing environmental impact. Fo